<a href="https://colab.research.google.com/github/Meghraj-Webllisto/language-translation/blob/master/17_02fairseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# working on prepairing training data fo coustom language chinese or japense or korean

In [1]:
cd /content/drive/My Drive/fairseq

/content/drive/My Drive/fairseq


In [2]:
ls

build/              fairseq/           interactive.py            scripts/
CODE_OF_CONDUCT.md  fairseq_cli/       iwslt17.de_fr.en.bpe16k/  setup.py
CONTRIBUTING.md     fairseq.egg-info/  iwslt17_orig/             tests/
dist/               fairseq.gif        LICENSE                   train.py
docs/               fairseq_logo.png   preprocess.py             validate.py
eval_lm.py          generate.py        README.md
examples/           hubconf.py         score.py


In [3]:
cd scripts 

/content/drive/My Drive/fairseq/scripts


In [4]:
ls

average_checkpoints.py  count_docs.py        split_train_valid_docs.py
build_sym_alignment.py  __init__.py          spm_decode.py
compare_namespaces.py   read_binarized.py    spm_encode.py
compound_split_bleu.sh  rm_pt.py             spm_train.py
convert_dictionary.lua  sacrebleu_pregen.sh  wav2vec_featurize.py
convert_model.lua       shard_docs.py        wav2vec_manifest.py


In [5]:
cd ..

/content/drive/My Drive/fairseq


In [6]:
pip install sentencepiece

     |████████████████████████████████| 1.0MB 2.7MB/s 


In [7]:
rm -r custom_orig

rm: cannot remove 'custom_orig': No such file or directory


In [0]:
TGT = en
Lang = en
for LANG in "ja" "en"; do
    cat "$ORIG/ja"-${TGT}.${LANG} \
        | grep -v '<url>' \
        | grep -v '<talkid>' \
        | grep -v '<keywords>' \
        | grep -v '<speaker>' \
        | grep -v '<reviewer' \
        | grep -v '<translator' \
        | grep -v '<doc' \
        | grep -v '</doc>' \
        | sed -e 's/<title>//g' \
        | sed -e 's/<\/title>//g' \
        | sed -e 's/<description>//g' \
        | sed -e 's/<\/description>//g' \
        | sed 's/^\s*//g' \
        | sed 's/\s*$//g' \
        > "$DATA/${SRC}-${TGT}.${LANG}"
done

In [20]:
pwd

'/content/drive/My Drive/fairseq'

In [35]:
%%bash
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

SRCS=(
    "en"
)
TGT=zh

ROOT=$(dirname "$0")
SCRIPTS=./scripts
SPM_TRAIN=$SCRIPTS/spm_train.py
SPM_ENCODE=$SCRIPTS/spm_encode.py

BPESIZE=16384
ORIG=$ROOT/custom_orig
DATA=$ROOT/custom.en-zh.en.bpe16k
mkdir -p "$ORIG" "$DATA"

TRAIN_MINLEN=1  # remove sentences with <1 BPE token
TRAIN_MAXLEN=250  # remove sentences with >250 BPE tokens

# URLS=(
#     "https://wit3.fbk.eu/archive/2017-01-trnted/texts/de/en/de-en.tgz"

#     "https://wit3.fbk.eu/archive/2017-01-trnted/texts/fr/en/fr-en.tgz"
# )
# ARCHIVES=(
#     "de-en.tgz"
#     "fr-en.tgz"
# )
# VALID_SETS=(
#     "IWSLT17.TED.dev2010.de-en IWSLT17.TED.tst2010.de-en IWSLT17.TED.tst2011.de-en IWSLT17.TED.tst2012.de-en IWSLT17.TED.tst2013.de-en IWSLT17.TED.tst2014.de-en IWSLT17.TED.tst2015.de-en"
#     "IWSLT17.TED.dev2010.fr-en IWSLT17.TED.tst2010.fr-en IWSLT17.TED.tst2011.fr-en IWSLT17.TED.tst2012.fr-en IWSLT17.TED.tst2013.fr-en IWSLT17.TED.tst2014.fr-en IWSLT17.TED.tst2015.fr-en"
# )

# download and extract data
# for ((i=0;i<${#URLS[@]};++i)); do
#     ARCHIVE=$ORIG/${ARCHIVES[i]}
#     if [ -f "$ARCHIVE" ]; then
#         echo "$ARCHIVE already exists, skipping download"
#     else
#         URL=${URLS[i]}
#         wget -P "$ORIG" "$URL"
#         if [ -f "$ARCHIVE" ]; then
#             echo "$URL successfully downloaded."
#         else
#             echo "$URL not successfully downloaded."
#             exit 1
#         fi
#     fi
#     FILE=${ARCHIVE: -4}
#     if [ -e "$FILE" ]; then
#         echo "$FILE already exists, skipping extraction"
#     else
#         tar -C "$ORIG" -xzvf "$ARCHIVE"
#     fi
# done

echo "pre-processing train data..."

for LANG in "zh" "en"; do
    cat "$ORIG/en"-${TGT}.${LANG} \
        | grep -v '<url>' \
        | grep -v '<talkid>' \
        | grep -v '<keywords>' \
        | grep -v '<speaker>' \
        | grep -v '<reviewer' \
        | grep -v '<translator' \
        | grep -v '<doc' \
        | grep -v '</doc>' \
        | sed -e 's/<title>//g' \
        | sed -e 's/<\/title>//g' \
        | sed -e 's/<description>//g' \
        | sed -e 's/<\/description>//g' \
        | sed 's/^\s*//g' \
        | sed 's/\s*$//g' \
        > "$DATA/train.en${SRC}-${TGT}.${LANG}"
done

# echo "pre-processing valid data..."
# for ((i=0;i<${#SRCS[@]};++i)); do
#     SRC=${SRCS[i]}
#     VALID_SET=(${VALID_SETS[i]})
#     for ((j=0;j<${#VALID_SET[@]};++j)); do
#         FILE=${VALID_SET[j]}
#         for LANG in "$SRC" "$TGT"; do
#             grep '<seg id' "$ORIG/${SRC}-${TGT}/${FILE}.${LANG}.xml" \
#                 | sed -e 's/<seg id="[0-9]*">\s*//g' \
#                 | sed -e 's/\s*<\/seg>\s*//g' \
#                 | sed -e "s/\’/\'/g" \
#                 > "$DATA/valid${j}.${SRC}-${TGT}.${LANG}"
#         done
#     done
# done

# learn BPE with sentencepiece
# TRAIN_FILES=$(for SRC in "${SRCS[@]}"; do \
# echo $DATA/train.${SRC}-${TGT}.${SRC}; \
# echo $DATA/train.${SRC}-${TGT}.${TGT}; done | tr "\n" ",")
# echo "learning joint BPE over ${TRAIN_FILES}..."
# python "$SPM_TRAIN" \
#     --input=$TRAIN_FILES \
#     --model_prefix=$DATA/sentencepiece.bpe \
#     --vocab_size=$BPESIZE \
#     --character_coverage=1.0 \
#     --model_type=bpe

# encode train/valid/test
echo "encoding train/valid with learned BPE..."
for SRC in "${SRCS[@]}"; do
    for LANG in "$SRC" "$TGT"; do
        python "$SPM_ENCODE" \
            --model "$DATA/sentencepiece.bpe.model" \
            --output_format=piece \
            --inputs $DATA/train.${SRC}-${TGT}.${SRC} $DATA/train.${SRC}-${TGT}.${TGT} \
            --outputs $DATA/train.bpe.${SRC}-${TGT}.${SRC} $DATA/train.bpe.${SRC}-${TGT}.${TGT} \
            --min-len $TRAIN_MINLEN --max-len $TRAIN_MAXLEN
        # python "$SPM_ENCODE" \
        #     --model "$DATA/sentencepiece.bpe.model" \
        #     --output_format=piece \
        #     --inputs $DATA/valid.${SRC}-${TGT}.${SRC} $DATA/valid.${SRC}-${TGT}.${TGT} \
        #     --outputs $DATA/valid.bpe.${SRC}-${TGT}.${SRC} $DATA/valid.bpe.${SRC}-${TGT}.${TGT}
    done
done

pre-processing train data...
encoding train/valid with learned BPE...


processed 10000 lines
processed 20000 lines
processed 30000 lines
processed 40000 lines
processed 50000 lines
processed 60000 lines
processed 70000 lines
skipped 49 empty lines
filtered 0 lines
processed 10000 lines
processed 20000 lines
processed 30000 lines
processed 40000 lines
processed 50000 lines
processed 60000 lines
processed 70000 lines
skipped 49 empty lines
filtered 0 lines


In [30]:
import sentencepiece as s
print(s.__file__)

/usr/local/lib/python3.6/dist-packages/sentencepiece.py


In [24]:
%%bash
#!/bin/bash
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

SRCS=(
    "de"
    "fr"
)
TGT=en

ROOT=$(dirname "$0")
SCRIPTS=$ROOT/scripts
SPM_TRAIN=$SCRIPTS/spm_train.py
SPM_ENCODE=$SCRIPTS/spm_encode.py

BPESIZE=16384
ORIG=$ROOT/iwslt17_orig
DATA=$ROOT/iwslt17.de_fr.en.bpe16k
mkdir -p "$ORIG" "$DATA"

TRAIN_MINLEN=1  # remove sentences with <1 BPE token
TRAIN_MAXLEN=250  # remove sentences with >250 BPE tokens

URLS=(
    "https://wit3.fbk.eu/archive/2017-01-trnted/texts/de/en/de-en.tgz"

    "https://wit3.fbk.eu/archive/2017-01-trnted/texts/fr/en/fr-en.tgz"
)
ARCHIVES=(
    "de-en.tgz"
    "fr-en.tgz"
)
VALID_SETS=(
    "IWSLT17.TED.dev2010.de-en IWSLT17.TED.tst2010.de-en IWSLT17.TED.tst2011.de-en IWSLT17.TED.tst2012.de-en IWSLT17.TED.tst2013.de-en IWSLT17.TED.tst2014.de-en IWSLT17.TED.tst2015.de-en"
    "IWSLT17.TED.dev2010.fr-en IWSLT17.TED.tst2010.fr-en IWSLT17.TED.tst2011.fr-en IWSLT17.TED.tst2012.fr-en IWSLT17.TED.tst2013.fr-en IWSLT17.TED.tst2014.fr-en IWSLT17.TED.tst2015.fr-en"
)

# download and extract data
for ((i=0;i<${#URLS[@]};++i)); do
    ARCHIVE=$ORIG/${ARCHIVES[i]}
    if [ -f "$ARCHIVE" ]; then
        echo "$ARCHIVE already exists, skipping download"
    else
        URL=${URLS[i]}
        wget -P "$ORIG" "$URL"
        if [ -f "$ARCHIVE" ]; then
            echo "$URL successfully downloaded."
        else
            echo "$URL not successfully downloaded."
            exit 1
        fi
    fi
    FILE=${ARCHIVE: -4}
    if [ -e "$FILE" ]; then
        echo "$FILE already exists, skipping extraction"
    else
        tar -C "$ORIG" -xzvf "$ARCHIVE"
    fi
done

echo "pre-processing train data..."
for SRC in "${SRCS[@]}"; do
    for LANG in "${SRC}" "${TGT}"; do
        cat "$ORIG/${SRC}-${TGT}/train.tags.${SRC}-${TGT}.${LANG}" \
            | grep -v '<url>' \
            | grep -v '<talkid>' \
            | grep -v '<keywords>' \
            | grep -v '<speaker>' \
            | grep -v '<reviewer' \
            | grep -v '<translator' \
            | grep -v '<doc' \
            | grep -v '</doc>' \
            | sed -e 's/<title>//g' \
            | sed -e 's/<\/title>//g' \
            | sed -e 's/<description>//g' \
            | sed -e 's/<\/description>//g' \
            | sed 's/^\s*//g' \
            | sed 's/\s*$//g' \
            > "$DATA/train.${SRC}-${TGT}.${LANG}"
    done
done

echo "pre-processing valid data..."
for ((i=0;i<${#SRCS[@]};++i)); do
    SRC=${SRCS[i]}
    VALID_SET=(${VALID_SETS[i]})
    for ((j=0;j<${#VALID_SET[@]};++j)); do
        FILE=${VALID_SET[j]}
        for LANG in "$SRC" "$TGT"; do
            grep '<seg id' "$ORIG/${SRC}-${TGT}/${FILE}.${LANG}.xml" \
                | sed -e 's/<seg id="[0-9]*">\s*//g' \
                | sed -e 's/\s*<\/seg>\s*//g' \
                | sed -e "s/\’/\'/g" \
                > "$DATA/valid${j}.${SRC}-${TGT}.${LANG}"
        done
    done
done

# learn BPE with sentencepiece
TRAIN_FILES=$(for SRC in "${SRCS[@]}"; do echo $DATA/train.${SRC}-${TGT}.${SRC}; echo $DATA/train.${SRC}-${TGT}.${TGT}; done | tr "\n" ",")
echo "learning joint BPE over ${TRAIN_FILES}..."
python "$SPM_TRAIN" \
    --input=$TRAIN_FILES \
    --model_prefix=$DATA/sentencepiece.bpe \
    --vocab_size=$BPESIZE \
    --character_coverage=1.0 \
    --model_type=bpe

# encode train/valid/test
echo "encoding train/valid with learned BPE..."
for SRC in "${SRCS[@]}"; do
    for LANG in "$SRC" "$TGT"; do
        python "$SPM_ENCODE" \
            --model "$DATA/sentencepiece.bpe.model" \
            --output_format=piece \
            --inputs $DATA/train.${SRC}-${TGT}.${SRC} $DATA/train.${SRC}-${TGT}.${TGT} \
            --outputs $DATA/train.bpe.${SRC}-${TGT}.${SRC} $DATA/train.bpe.${SRC}-${TGT}.${TGT} \
            --min-len $TRAIN_MINLEN --max-len $TRAIN_MAXLEN
        python "$SPM_ENCODE" \
            --model "$DATA/sentencepiece.bpe.model" \
            --output_format=piece \
            --inputs $DATA/valid0.${SRC}-${TGT}.${SRC} $DATA/valid0.${SRC}-${TGT}.${TGT} \
            --outputs $DATA/valid0.bpe.${SRC}-${TGT}.${SRC} $DATA/valid0.bpe.${SRC}-${TGT}.${TGT}
    done
done

Process is interrupted.
